In [36]:
import requests
import tarfile
import os

import numpy as np
import pandas as pd
import plotly.express as px
from openpyxl.utils.units import points_to_pixels

In [37]:
def download_and_extract(url: str, extract_to: str):
    """
    Downloads a tar.gz file from a URL and extracts it to a directory.
    Args:
    - url (str): URL of the tar.gz file to download.
    - extract_to (str): Directory path to extract the contents of the tar.gz file.
    """
    # Get the filename from the URL
    filename = url.split('/')[-1]

    # Download the file
    print("Downloading the file...")
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.raw.read())
        print("Download completed.")
    else:
        print("Failed to download the file.")
        return

    # Extract the tar.gz file
    print("Extracting the file...")
    try:
        with tarfile.open(filename, 'r:gz') as tar:
            tar.extractall(path=extract_to)
        print("Extraction completed.")
    except Exception as e:
        print(f"Failed to extract the file: {e}")
    finally:
        # Optionally remove the tar.gz file after extraction
        os.remove(filename)
        print("Downloaded tar.gz file removed.")

# URL of the tar.gz file
url = "https://video.udacity-data.com/topher/2024/August/66b9ba05_arvato_data.tar/arvato_data.tar.gz"

# Call the function with the URL
# download_and_extract(url, extract_to=".")

# Part 0: Get to Know the Data

In [38]:
population = pd.read_csv("../data/Udacity_AZDIAS_052018.csv", sep=";", nrows=100000)
customers = pd.read_csv("../data/Udacity_CUSTOMERS_052018.csv", sep=";", nrows=10000)

population.columns = population.columns.str.lower()
customers.columns = customers.columns.str.lower()

/tmp/ipykernel_62336/2617753861.py:1: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  population = pd.read_csv("../data/Udacity_AZDIAS_052018.csv", sep=";", nrows=100000)
/tmp/ipykernel_62336/2617753861.py:2: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  customers = pd.read_csv("../data/Udacity_CUSTOMERS_052018.csv", sep=";", nrows=10000)


In [39]:
meta = pd.read_excel("../data/meta/dias_values.xlsx", header=1)
meta.columns = meta.columns.str.lower()
meta.drop(columns="unnamed: 0", inplace=True)

meta[["attribute", "description"]] = meta[["attribute", "description"]].ffill()
meta["attribute"] = meta["attribute"].str.lower()

meta["meaning"] = meta["meaning"].ffill()

meta["value"] = [v.split(", ") if type(v) == str and "," in v else v for v in meta["value"]]
meta = meta.explode("value")

## Data Types

In [40]:
population.dtypes.value_counts()

float64    267
int64       93
object       6
Name: count, dtype: int64

### Strings
Besides numeric values there are strings in the datasets which need a separate preprocessing:
- `CAMEO_DEU_2015`, `CAMEO_DEUG_2015` and `CAMEO_INTL_2015`
  - have `X` or `XX` values that should be treated as null values
  - partially have numeric values as strings that should be unified
  - are classifications that should be one-hot-encoded
    - `CAMEO_INTL_2015` values are a composition of household and family type and should be separated
- `D19_LETZTER_KAUF_BRANCHE` shows the sector of the last purchase which is ineligible for our segmentation report and model
- `EINGEFUEGT_AM` is the timestamp where the data was created in the database and also unnecessary
- `OST_WEST_KZ` is a flag labeled as `W` or `O` that should be one-hot-encoded

In [41]:
population.select_dtypes(include="object").head()

,cameo_deu_2015,cameo_deug_2015,cameo_intl_2015,d19_letzter_kauf_branche,eingefuegt_am,ost_west_kz
0,NaN,NaN,NaN,NaN,NaN,NaN
1,8A,8.0,51.0,NaN,1992-02-10 00:00:00,W
2,4C,4.0,24.0,D19_UNBEKANNT,1992-02-12 00:00:00,W
3,2A,2.0,12.0,D19_UNBEKANNT,1997-04-21 00:00:00,W
4,6B,6.0,43.0,D19_SCHUHE,1992-02-12 00:00:00,W


In [42]:
population["cameo_deu_2015"].unique()

array([nan, '8A', '4C', '2A', '6B', '8C', '4A', '2D', '1A', '1E', '9D',
       '5C', '8B', '7A', '5D', '9E', '9B', '1B', '3D', '4E', '4B', '3C',
       '5A', '7B', '9A', '6D', '6E', '2C', '7C', '9C', '7D', '5E', '1D',
       '8D', '6C', '6A', '5B', '4D', '3A', '2B', '7E', '3B', '6F', '5F',
       '1C', 'XX'], dtype=object)

In [43]:
population["cameo_deug_2015"].unique()

array([nan, 8.0, 4.0, 2.0, 6.0, 1.0, 9.0, 5.0, 7.0, 3.0, '4', '3', '7',
       '2', '8', '9', '6', '5', '1', 'X'], dtype=object)

In [44]:
population["cameo_intl_2015"].unique()

array([nan, 51.0, 24.0, 12.0, 43.0, 54.0, 22.0, 14.0, 13.0, 15.0, 33.0,
       41.0, 34.0, 55.0, 25.0, 23.0, 31.0, 52.0, 35.0, 45.0, 44.0, 32.0,
       '22', '24', '41', '12', '54', '51', '44', '35', '23', '25', '14',
       '34', '52', '55', '31', '32', '15', '13', '43', '33', '45', 'XX'],
      dtype=object)

In [45]:
meta[meta["attribute"] == "cameo_intl_2015"].head(10)

,attribute,description,value,meaning
105,cameo_intl_2015,CAMEO classification 2015 - international typo...,-1,unknown
106,cameo_intl_2015,(each German CAMEO code belongs to one interna...,11,Wealthy Households-Pre-Family Couples & Singles
107,cameo_intl_2015,(each German CAMEO code belongs to one interna...,12,Wealthy Households-Young Couples With Children
108,cameo_intl_2015,(each German CAMEO code belongs to one interna...,13,Wealthy Households-Families With School Age Ch...
109,cameo_intl_2015,(each German CAMEO code belongs to one interna...,14,Wealthy Households-Older Families & Mature Co...
110,cameo_intl_2015,(each German CAMEO code belongs to one interna...,15,Wealthy Households-Elders In Retirement
111,cameo_intl_2015,(each German CAMEO code belongs to one interna...,21,Prosperous Households-Pre-Family Couples & Sin...
112,cameo_intl_2015,(each German CAMEO code belongs to one interna...,22,Prosperous Households-Young Couples With Children
113,cameo_intl_2015,(each German CAMEO code belongs to one interna...,23,Prosperous Households-Families With School Age...
114,cameo_intl_2015,(each German CAMEO code belongs to one interna...,24,Prosperous Households-Older Families & Mature ...


In [46]:
population["ost_west_kz"].unique()

array([nan, 'W', 'O'], dtype=object)

In [47]:
population = population.drop(columns=["d19_letzter_kauf_branche", "eingefuegt_am"])

In [48]:
def prepare_cameo_classifications(df: pd.DataFrame) -> pd.DataFrame:

    cameo_columns = df.columns[df.columns.str.startswith("cameo")]
    df[cameo_columns] = df[cameo_columns].replace(["X", "XX"], np.nan)

    for cameo_column in cameo_columns:
        if cameo_column != "cameo_deu_2015":
            df[cameo_column] = df[cameo_column].astype(float)

    df["cameo_intl_2015_household"] = [str(v)[0] if v != -1 and pd.notnull(v) else v for v in df["cameo_intl_2015"]]
    df["cameo_intl_2015_family"] = [str(v)[1] if v != -1 and pd.notnull(v) else v for v in df["cameo_intl_2015"]]
    df = df.drop(columns=["cameo_intl_2015"])

    return df

population = prepare_cameo_classifications(population)

### Unified Numerical Types
All numerical values in the dataset are actually integers, but Pandas converted most of them to float to be able to handle null values.

Furthermore, most of the numerical columns are actually categorical variables, which will be addressed with one-hot-encoding at the end of the pre-processing. However, we will start by converting all integer values to floats, because it simplifies the initial preprocessing.

In [49]:
int_cols = population.select_dtypes(include=int).columns
population[int_cols] = population[int_cols].astype(float)

## Missing Values
Before analyzing the missing values we start by converting unknown and invalid values to null in order to be able to handle them as missing data.

### Unknown Values
The provided list that explains the meanings of the numeric attribute values can be used to identify unknown values.

#### Missing Meta Attributes
- some attribute names have to be rectified to map them with the population dataset
- but these attributes from the meta list couldn't be assigned/found in the population dataset: `bip_flag`, `geoscore_kls7`, `haushaltsstruktur`, `wachstumsgebiet_nb`


In [50]:
meta[meta["meaning"].str.contains("unknown") | meta["meaning"].str.startswith("no transaction")]

,attribute,description,value,meaning
0,ager_typ,best-ager typology,-1,unknown
5,alterskategorie_grob,age classification through prename analysis,-1,unknown
5,alterskategorie_grob,age classification through prename analysis,0,unknown
11,alter_hh,main age within the household,0,unknown / no main age detectable
33,anrede_kz,gender,-1,unknown
...,...,...,...,...
2238,wachstumsgebiet_nb,growing area (population growth in the last 5 ...,0,unknown
2244,w_keit_kind_hh,likelihood of a child present in this household,-1,unknown
2244,w_keit_kind_hh,likelihood of a child present in this household,0,unknown
2251,zabeotyp,typification of energy consumers,-1,unknown


In [51]:
def identify_missing_meta_attributes(meta: pd.DataFrame, df: pd.DataFrame) -> pd.DataFrame:
    missing_meta_attributes = list(set(meta["attribute"].unique()).difference(df.columns))
    missing_meta_attributes.sort()
    missing_meta_attributes.remove("cameo_intl_2015")
    return missing_meta_attributes

In [52]:
identify_missing_meta_attributes(meta, population)

['bip_flag',
 'd19_banken_direkt_rz',
 'd19_banken_gross_rz',
 'd19_banken_lokal_rz',
 'd19_banken_rest_rz',
 'd19_bekleidung_geh_rz',
 'd19_bekleidung_rest_rz',
 'd19_bildung_rz',
 'd19_bio_oeko_rz',
 'd19_buch_rz',
 'd19_digit_serv_rz',
 'd19_drogerieartikel_rz',
 'd19_energie_rz',
 'd19_freizeit_rz',
 'd19_garten_rz',
 'd19_handwerk_rz',
 'd19_haus_deko_rz',
 'd19_kinderartikel_rz',
 'd19_kk_kundentyp',
 'd19_kosmetik_rz',
 'd19_lebensmittel_rz',
 'd19_lotto_rz',
 'd19_nahrungsergaenzung_rz',
 'd19_ratgeber_rz',
 'd19_reisen_rz',
 'd19_sammelartikel_rz',
 'd19_schuhe_rz',
 'd19_sonstige_rz',
 'd19_technik_rz',
 'd19_telko_mobile_rz',
 'd19_telko_rest_rz',
 'd19_tierartikel_rz',
 'd19_versand_rest_rz',
 'd19_versicherungen_rz',
 'd19_vollsortiment_rz',
 'd19_wein_feinkost_rz',
 'geoscore_kls7',
 'haushaltsstruktur',
 'kba13_ccm_1400_2500',
 'soho_flag',
 'wachstumsgebiet_nb']

In [53]:
def rectify_meta_attributes(meta: pd.DataFrame) -> pd.DataFrame:
    meta["attribute"] = meta["attribute"].replace(r"_rz$", "", regex=True)

    attribute_renaming = {
        "d19_buch": "d19_buch_cd",
        "d19_kk_kundentyp": "kk_kundentyp",
        "kba13_ccm_1400_2500": "kba13_ccm_1401_2500",
        "soho_flag": "soho_kz"
    }

    meta["attribute"] = meta["attribute"].replace(attribute_renaming)

    return meta

meta = rectify_meta_attributes(meta)

In [73]:
identify_missing_meta_attributes(meta, population)

['bip_flag', 'geoscore_kls7', 'haushaltsstruktur', 'wachstumsgebiet_nb']

In [54]:
unknown_values = meta[meta["meaning"].str.contains("unknown") | meta["meaning"].str.startswith("no transaction")]

In [55]:
def convert_unknown_values_to_null(df: pd.DataFrame, meta: pd.DataFrame) -> pd.DataFrame:
    unknown_values = meta[meta["meaning"].str.contains("unknown") | meta["meaning"].str.startswith("no transaction")]

    for attribute, unknown_value in zip(unknown_values["attribute"], unknown_values["value"]):
        if attribute in df:
            df[attribute] = df[attribute].replace(float(unknown_value), np.nan)

    return df

population = convert_unknown_values_to_null(population, meta)

#### Population Attributes without Meta Information
- 54 columns from the population dataset are not included in the provided meta information
  - xxx
  - we will keep them anyway to see if they are valueable insights for the clustering or predictive model
- the additional `d19` columns will be handled like the other ones, so 0 and 10 will be assumed as null values
- there are no remaining -1 values found, where the chance is high that they represent unknown values
- `lnr` is an identifier for the dataset records and should be removed

In [56]:
missing_population_attributes = list(set(population.columns).difference(meta["attribute"].unique()))
missing_population_attributes.sort()
missing_population_attributes.remove("cameo_intl_2015_household")
missing_population_attributes.remove("cameo_intl_2015_family")

population[missing_population_attributes].describe()

,akt_dat_kl,alter_kind1,alter_kind2,alter_kind3,alter_kind4,alterskategorie_fein,anz_kinder,anz_statistische_haushalte,arbeit,cjt_katalognutzer,...,strukturtyp,umfeld_alt,umfeld_jung,ungleichenn_flag,verdichtungsraum,vha,vhn,vk_dht4a,vk_distanz,vk_zg11
count,91615.000000,9092.000000,3269.000000,690.000000,127.000000,70204.000000,91615.000000,89474.000000,89006.000000,99456.000000,...,89004.000000,88973.000000,88973.000000,91615.000000,89004.000000,91615.000000,86591.000000,91324.000000,91324.000000,91324.000000
mean,4.456170,11.771887,13.392781,14.455072,14.937008,13.720372,0.153785,7.618604,3.163158,3.325782,...,2.549897,3.228137,4.007868,0.091361,4.653487,0.444894,2.423612,6.022732,7.566357,5.977114
std,3.644775,4.107494,3.289373,2.759101,2.653982,5.068927,0.500315,14.253304,0.998052,1.497127,...,0.748359,1.249616,1.115011,0.288123,8.531882,1.154330,1.168891,2.854200,3.247799,2.775939
min,1.000000,2.000000,4.000000,5.000000,8.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,1.000000,8.000000,11.000000,13.000000,13.000000,11.000000,0.000000,1.000000,3.000000,2.000000,...,2.000000,2.000000,3.000000,0.000000,0.000000,0.000000,2.000000,4.000000,6.000000,4.000000
50%,3.000000,12.000000,14.000000,15.000000,15.000000,14.000000,0.000000,4.000000,3.000000,4.000000,...,3.000000,3.000000,4.000000,0.000000,1.000000,0.000000,2.000000,6.000000,8.000000,6.000000
75%,9.000000,15.000000,16.000000,17.000000,17.000000,17.000000,0.000000,9.000000,4.000000,5.000000,...,3.000000,4.000000,5.000000,0.000000,5.000000,0.000000,3.000000,9.000000,10.000000,8.000000
max,9.000000,18.000000,18.000000,18.000000,18.000000,25.000000,9.000000,367.000000,9.000000,5.000000,...,3.000000,5.000000,5.000000,1.000000,45.000000,5.000000,4.000000,11.000000,13.000000,11.000000


In [57]:
population = population.drop(columns=["lnr"])

In [58]:
def convert_unknown_d19_values_to_null(df: pd.DataFrame) -> pd.DataFrame:
    d19_columns = df.columns[df.columns.str.startswith("d19_")]
    df[d19_columns] = df[d19_columns].replace([0.0, 10.0], np.nan)
    return df

population = convert_unknown_d19_values_to_null(population)

### Invalid Values
#### Counts & Dates
Some counts and dates are logically invalid and should be replaced with null values:
- `geburtsjahr` is the year of birth and has invalid 0 values
- `anz_personen` are the number of adult persons in the household and has invalid 0 values although e.g. the year of birth shows that there are indeed adults

In [59]:
meta[meta["meaning"].str.contains("numeric")]

,attribute,description,value,meaning
36,anz_haushalte_aktiv,number of households in the building,…,numeric value (typically coded from 1-10)
37,anz_hh_titel,number of academic title holder in building,…,numeric value (typically coded from 1-10)
38,anz_personen,number of adult persons in the household,…,numeric value (typically coded from 1-3)
39,anz_titel,number of professional title holder in household,…,numeric value (typically coded from 1-10)
711,geburtsjahr,year of birth,…,numeric value
1167,kba13_anzahl_pkw,number of cars in the PLZ8,…,numeric value
1986,min_gebaeudejahr,year the building was first mentioned in our d...,…,numeric value


In [60]:
year_columns = population.columns[population.columns.str.contains("jahr")]

population[year_columns].describe()

,eingezogenam_hh_jahr,geburtsjahr,min_gebaeudejahr,praegende_jugendjahre
count,91615.000000,100000.000000,89474.000000,87688.000000
mean,2003.723386,1095.000700,1993.180242,9.286812
std,7.055596,977.344266,3.151232,4.017162
min,1986.000000,0.000000,1985.000000,1.000000
25%,1997.000000,0.000000,1992.000000,6.000000
50%,2003.000000,1943.000000,1992.000000,9.000000
75%,2010.000000,1970.000000,1993.000000,14.000000
max,2018.000000,2017.000000,2016.000000,15.000000


In [61]:
population["geburtsjahr"].value_counts()

geburtsjahr
0.0       44336
1965.0     1283
1967.0     1252
1968.0     1220
1970.0     1212
          ...  
1911.0        3
1912.0        3
1908.0        1
1904.0        1
1906.0        1
Name: count, Length: 113, dtype: int64

In [62]:
count_columns = population.columns[population.columns.str.startswith("anz_") | population.columns.str.contains("anzahl")]

population[count_columns].describe()

,anz_haushalte_aktiv,anz_hh_titel,anz_kinder,anz_personen,anz_statistische_haushalte,anz_titel,kba13_anzahl_pkw
count,89474.000000,89088.000000,91615.000000,91615.000000,89474.000000,91615.000000,88133.000000
mean,8.293918,0.040252,0.153785,1.728483,7.618604,0.004333,621.744670
std,15.220064,0.305668,0.500315,1.169732,14.253304,0.068771,339.952138
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,1.000000,1.000000,0.000000,386.000000
50%,4.000000,0.000000,0.000000,1.000000,4.000000,0.000000,551.000000
75%,10.000000,0.000000,0.000000,2.000000,9.000000,0.000000,780.000000
max,377.000000,15.000000,9.000000,45.000000,367.000000,2.000000,2300.000000


In [63]:
population[population["anz_personen"] == 0]["geburtsjahr"]

3        1957.0
16       1988.0
25       1991.0
64       1975.0
80       1974.0
          ...  
99864    1981.0
99895    1923.0
99929    1967.0
99934    1936.0
99943    1949.0
Name: geburtsjahr, Length: 3817, dtype: float64

#### Out of Meta Data Range
Some values are not included in the ranges from the given meta information, we will therefore handle them as null values.

In [64]:
def identify_invalid_oor_values(meta: pd.DataFrame, df: pd.DataFrame, log: bool = False) -> pd.DataFrame:
    categorical_values = meta[meta["value"] != "…"][["attribute", "value"]]
    categorical_values = categorical_values.rename(columns={"value": "meta_value"})

    invalid_oor_values = []

    for attribute in categorical_values["attribute"].unique():
        if attribute in df:
            pop_vc = df[attribute].value_counts().reset_index()
            pop_vc = pop_vc.merge(
                categorical_values[categorical_values["attribute"] == attribute]["meta_value"],
                left_on=attribute,
                right_on="meta_value",
                how="left",
            )

            if sum(pop_vc["meta_value"].isnull()) > 0:
                for out_of_range_value in pop_vc[pop_vc["meta_value"].isnull()].itertuples():
                    if log:
                        print(f"attribute: {attribute}, invalid value: {out_of_range_value[1]}, affected rows: {out_of_range_value.count}")
                    invalid_oor_values.append([attribute, out_of_range_value[1]])

    return pd.DataFrame(data=invalid_oor_values, columns=["attribute", "value"])


_ = identify_invalid_oor_values(meta, population, log=True)

attribute: kba05_modtemp, invalid value: 6.0, affected rows: 1077
attribute: lp_familie_fein, invalid value: 0.0, affected rows: 8329
attribute: lp_familie_grob, invalid value: 0.0, affected rows: 8329
attribute: lp_lebensphase_fein, invalid value: 0.0, affected rows: 10650
attribute: lp_lebensphase_grob, invalid value: 0.0, affected rows: 10307
attribute: ortsgr_kls9, invalid value: 0.0, affected rows: 2


In [65]:
def convert_invalid_values_to_null(df: pd.DataFrame) -> pd.DataFrame:
    df.loc[df["geburtsjahr"] == 0, "geburtsjahr"] = np.nan
    df.loc[df["anz_personen"] == 0, "anz_personen"] = np.nan

    invalid_oor_values = identify_invalid_oor_values(meta, population)
    for attribute, invalid_oor_value in zip(invalid_oor_values["attribute"], invalid_oor_values["value"]):
        if attribute in df:
            df[attribute] = df[attribute].replace(invalid_oor_value, np.nan)

    return df

population = convert_invalid_values_to_null(population)

### Remove Rows


In [66]:
population.isnull().sum(axis=1).sort_values(ascending=False)

54       321
48       321
14       321
77134    321
24       321
        ... 
35249     18
74262     17
41931     17
88395     17
96909     15
Length: 100000, dtype: int64

In [70]:
population.loc[54]

ager_typ                     NaN
akt_dat_kl                   NaN
alter_hh                     NaN
alter_kind1                  NaN
alter_kind2                  NaN
                            ... 
zabeotyp                     3.0
anrede_kz                    1.0
alterskategorie_grob         3.0
cameo_intl_2015_household    NaN
cameo_intl_2015_family       NaN
Name: 54, Length: 364, dtype: object

### Impute or Remove Columns
- `alter_kind1` to `alter_kind4` are largely null because most of the persons in the dataset haven't kids

In [68]:
missing_values_share = pd.Series(round(population.isnull().sum() / len(population) * 100, 1).sort_values(ascending=False), name="missing_values_share")

In [69]:
missing_values_share[missing_values_share > 0]

d19_versi_online_quote_12     100.0
d19_telko_online_quote_12     100.0
d19_banken_online_quote_12     99.9
alter_kind4                    99.9
titel_kz                       99.8
                              ...  
cjt_typ_2                       0.5
cjt_typ_6                       0.5
cjt_typ_5                       0.5
cjt_typ_1                       0.5
cjt_typ_4                       0.5
Name: missing_values_share, Length: 337, dtype: float64